# Import libs and define functions

In [139]:
import pandas as pd
import subprocess
import os
import shutil
import re
import sys 

In [140]:
def remove_end_in_name(filename):
    splitted = filename.split("_")
    splitted.pop()
    joined_string = "_".join(splitted)
    return joined_string

In [141]:
def get_creation_time(file_path):
    try:
        creation_time = os.path.getctime(file_path)
        return pd.to_datetime(creation_time, unit='s')
    except Exception as e:
        print(f"Error getting creation time for {file_path}: {e}")
        return None

In [142]:
def get_the_table_data():
    for f in os.listdir():
        if f.endswith(".csv"):
                video_schnitt_df = pd.read_csv(f)
                print(f"Table {f} loaded succesfully")
                return video_schnitt_df
    raise ValueError("There is no csv in this directory")

In [143]:
def get_video_duration(file_path):
    command = ['ffmpeg', '-i', file_path]
    result = subprocess.run(command, text=True, capture_output=True)
    output_lines = result.stderr.split('\n')
    duration_line = [line for line in output_lines if 'Duration' in line][0]
    duration = duration_line.strip().split(",")[0].split(" ")[1]
    return duration

In [144]:
def clean_the_data(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.strip()
    df = df.astype(str)
    print("Table Columns cleaned")
    return df

In [145]:
def cut_head_tail(original_video,output_file,cut_head="00:00:00",cut_tail="00:50:00"):

    # Construct the command
    command = [
        'ffmpeg',
        '-i', original_video,
        '-ss', cut_head,
        '-to', cut_tail,
        '-c:v','copy', #libx264 to encode and copy to test
        '-c:a', 'copy',
        output_file
    ]
    
     # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [146]:
def create_folder(folder_name):
    try:
        # Create a new folder in the current working directory
        os.mkdir(folder_name)
        print(f"Folder '{folder_name}' created successfully.")
    except FileExistsError:
        print(f"Folder '{folder_name}' already exists.")

In [147]:
def get_standbild(input_file,output_file,cut_head):
    
    #creat timedelta and add 1 miliseconds
    time_delta = pd.to_timedelta(cut_head)
    new_time_delta = time_delta + pd.Timedelta(milliseconds=1)

    #create dummy for calculation to transform data type back to datetype
    dummy_date = pd.Timestamp('1900-01-01')
    new_timestamp = dummy_date + new_time_delta

    #get the new time back to wished str format
    cut_tail = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

    # Construct the command
    command = [
        'ffmpeg',
        '-i', input_file,
        '-ss', cut_head,
        '-to', cut_tail,
        '-an', #audio no
        '-c:v', 'libx264', output_file # libx264 to encode and copy to test (but that didnt work the last time)
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [148]:
def get_audio(input_file,output_file,cut_head,cut_tail):
    
    #Construct command
    command = ['ffmpeg',
               '-i', input_file,
               '-ss', cut_head,
               '-to', cut_tail,
               '-vn',
               '-c:a', 'copy', output_file]
    
    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [149]:
def merge_audio_and_video(video_file,audio_file,output_file):
    #Construct command
    command = ['ffmpeg',
               '-i', video_file,
               '-i', audio_file,
               '-c:v', 'copy',
               '-c:a', 'copy', output_file
               ]
    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [150]:
def get_columns_to_cut(df):
    pattern = re.compile(r'^rausschneiden\d+_[a-z]+$')
    dummy = []
    for column in df.columns:
        if pattern.match(column):
            dummy.append(column)
    return dummy

In [151]:
def get_list_of_evens(input_list):
    dummy = []
    for i in range(len(input_list)):
        if i % 2 == 0:
            dummy.append(i)
    return dummy

In [152]:
def concatenate_videos(input_file, output_file):
    
    command = [
        'ffmpeg',
        '-f', 'concat', 
        '-i', input_file,
         '-c:v', 'copy', #libx264 to encode if needed
         '-c:a', 'copy',
        output_file
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print("An error occured:", e)

In [153]:
def check_for_non_zero_digit(string):
    # Define a regular expression pattern to match any digit other than 0
    pattern = re.compile('[1-9]')

    if pattern.search(string):
        return True       
    else:
        return False


In [154]:
def extract_data_from_row(df,row_index,columns):
    dic = df.loc[row_index,columns].to_dict()

    return dic

In [155]:
def check_special_case(df):
    columns = get_columns_to_cut(video_schnitt_df)
    for idx, video_name in df["dateiname"].items():
        data = video_schnitt_df["rausschneiden1_ab"][idx]
        if check_for_non_zero_digit(data) == True:
            dic = extract_data_from_row(df,idx,columns)
            counter = 0
            for key,value in dic.items():
                if value == "nan":
                    counter += 1
            if counter < 2:
                print(f"""Error because in row of {video_name}
we need to have at least 2 free 'rausschneiden...' columns 
to execute the code. Please change the csv as needed and
restart the script""")
                sys.exit("Script terminated due to error.")

In [156]:
def solve_special_case(df):
    
    for idx,video_name in df["dateiname"].items():
        
        if check_for_non_zero_digit(df["rausschneiden1_ab"][idx]) == False:
            continue

        else:
            columns = get_columns_to_cut(df)
            dic = extract_data_from_row(df,idx,columns)
            dummy = []
            counter = 0

            # put values from row into list and change first two columns to 000000
            for key, value in dic.items():
                if value != "nan":
                    dummy.append(value)
                if counter < 2:
                    dic[key] = "00:00:00"
                    counter += 1
        # fülle das jeweilige dic solange mit daten bis liste leer
        for key, value in dic.items():
            if value != "00:00:00" and dummy:
                dic[key] = dummy.pop(0)
        
        for key, value in dic.items():
            df.loc[idx, key] = value
    
    return df

In [157]:
def get_columns_to_divide(df):
    pattern = re.compile(r'^teilen\d+_[a-z]+$')
    dummy = []
    for column in df.columns:
        if pattern.match(column):
            dummy.append(column)
    return dummy

In [158]:
def calculate_time(*timestamps,kind="addition"):
    """
    Calculate time based on the given timestamps and operation type.

    Parameters:
    - timestamps (str): Variable number of timestamps.
    - kind (str): The operation type. Allowed values are 'addition' and 'subtraction'.
    """
    
    #addition
    if kind == "addition":
        stamp1 = None
        for timestamp in timestamps:
        
        # calculate timestamp
            if stamp1 is None:
                stamp1 = timestamp

            else:
                timedelta = stamp1 + pd.to_timedelta(timestamp)
                dummy_date = pd.Timestamp('1900-01-01')
                new_timestamp = dummy_date + timedelta
                timestamp = new_timestamp.strftime('%H:%M:%S.%f')[:-3]
                return timestamp
    
    #subtraction
    elif kind == "subtraction":
        stamp1 = None
        for timestamp in timestamps:
        
        # calculate timestamp
            if stamp1 is None:
                stamp1 = timestamp

            else:
                timedelta = stamp1 - pd.to_timedelta(timestamp)
                dummy_date = pd.Timestamp('1900-01-01')
                new_timestamp = dummy_date + timedelta
                timestamp = new_timestamp.strftime('%H:%M:%S.%f')[:-3]
                return timestamp        

In [159]:
def cut_in_2_pieces(input_file, timestamp, head_output, tail_output):
    
    # -ss = ab dort; -to = bis dort
    head_cmd = ['ffmpeg', '-i', input_file, '-to', timestamp, '-c:v', 'libx264', '-c:a', 'copy', head_output]
    tail_cmd = ['ffmpeg', '-i', input_file, '-ss', timestamp, '-c:v', 'libx264', '-c:a', 'copy', tail_output]

# Execute the commands
    try:
        subprocess.run(head_cmd, check=True)
        subprocess.run(tail_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

# Get file and clean it

In [209]:
video_schnitt_df = get_the_table_data()
video_schnitt_df = clean_the_data(video_schnitt_df)


Table Videoschnitt_Liste - Spaltennamen (1).csv loaded succesfully
Table Columns cleaned


# Check Special Cases and input error

In [161]:
check_special_case(video_schnitt_df)

In [162]:
solve_special_case(video_schnitt_df)

,dateiname,notiz,rausschneiden1_ab,rausschneiden1_bis,rausschneiden2_ab,rausschneiden2_bis,rausschneiden3_ab,rausschneiden3_bis,rausschneiden4_ab,rausschneiden4_bis,...,rausschneiden5_bis,rausschneiden6_ab,rausschneiden6_bis,rausschneiden7_ab,rausschneiden7_bis,unnamed:_16,standbild_bis,teilen1_bei,teilen2_bei,teilen3_bei
0,Normalverteilung.mp4,nan,00:00:00,00:00:00,00:02:00,00:03:00,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,00:01:00,00:05:00,nan,nan
1,ELR_Geradengleichung.mp4,nan,nan,00:01:44.000,00:02:04.000,00:02:08.000,00:10:35.500,00:10:39.500,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,00:02:00,00:03:00,nan
2,ELR_Geradengleichung_Einleitung.mp4,nan,00:00:00,00:00:00,00:02:00,00:05:00,00:06:00,00:07:00,00:08:00,nan,...,nan,nan,nan,nan,nan,nan,nan,00:02:00,00:03:00,00:05:00
3,Normalverteilung2.mp4,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,00:01:00,nan,nan,nan
4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


# Get Standbild

In [163]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
        
        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue
        else:
            cut_head = video_schnitt_df["standbild_bis"][idx]
        
        output_file = f"{video_name.split('.')[0]}_standbild.mp4"

        print("video_name:",video_name,"cuttime:",cut_head,)
        get_standbild(video_name,output_file,cut_head)
except Exception as error:
    print("An error occurred:", error)

video_name: Normalverteilung.mp4 cuttime: 00:01:00


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

video_name: Normalverteilung2.mp4 cuttime: 00:01:00


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

[libx264 @ 0x7f8462f23080] using SAR=1/1
[libx264 @ 0x7f8462f23080] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x7f8462f23080] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x7f8462f23080] 264 - core 164 r3108 31e19f9 - H.264/MPEG-4 AVC codec - Copyleft 2003-2023 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=6 lookahead_threads=1 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to 'Normalverteilung2_standbild.mp4':
  Metadata:
    major_brand     : isom
    minor_vers

# Get audio

In [164]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
        
        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue

        #Define arguments
        output_file = f"{video_name.split('.')[0]}_head_audio.mp4"
        cut_head = "00:00:00"
        cut_tail = video_schnitt_df["standbild_bis"][idx]

        print("video_name",video_name,"output_file", output_file,"cut_head",cut_head,"cut_tail:", cut_tail)
        get_audio(video_name,output_file,cut_head,cut_tail)

except Exception as error:
    print("An error occurred:", error)

video_name Normalverteilung.mp4 output_file Normalverteilung_head_audio.mp4 cut_head 00:00:00 cut_tail: 00:01:00


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

video_name Normalverteilung2.mp4 output_file Normalverteilung2_head_audio.mp4 cut_head 00:00:00 cut_tail: 00:01:00


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

# Cut head of the original Video

In [165]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue
        
        # define arguments
        cut_head = video_schnitt_df["standbild_bis"][idx]
        cut_tail = get_video_duration(video_name)
        output_file = f"{video_name.split('.')[0]}_ohne_start_standbild.mp4"
    
        print("input name:",video_name,"Schnittpunkte:", cut_head, cut_tail)
        cut_head_tail(video_name, output_file, cut_head, cut_tail)
except Exception as error:
    print("An error occurred:", error)

input name: Normalverteilung.mp4 Schnittpunkte: 00:01:00 00:50:46.08


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

input name: Normalverteilung2.mp4 Schnittpunkte: 00:01:00 00:50:46.13


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

# Merge audio and video

In [166]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue
        
        video_file = f"{video_name.split('.')[0]}_standbild.mp4"
        audio_file = f"{video_name.split('.')[0]}_head_audio.mp4"
        output_file = f"{video_name.split('.')[0]}_merged_start.mp4"
        
        print("video:", video_file, "audio:", audio_file, "output", output_file)
        merge_audio_and_video(video_file,audio_file,output_file)

except Exception as error:
    print("An error occured:", error)

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

video: Normalverteilung_standbild.mp4 audio: Normalverteilung_head_audio.mp4 output Normalverteilung_merged_start.mp4


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'Normalverteilung_standbild.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf60.3.100
  Duration: 00:00:00.03, start: 0.000000, bitrate: 3612 kb/s
  Stream #0:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt709, progressive), 1920x1080 [SAR 1:1 DAR 16:9], 3405 kb/s, 30 fps, 30 tbr, 15360 tbn (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc60.3.100 libx264
Input #1, mov,mp4,m4a,3gp,3g2,mj2, from 'Normalverteilung_head_audio.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2mp41
    encoder         : Lavf60.3.100
  Duration: 00:01:00.02, start: 0.000000, bitrate: 193 kb/s
  Stream #1:0[0x1](und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 191 kb/s (default)
    Metadata:
      handler_name    : Sou

video: Normalverteilung2_standbild.mp4 audio: Normalverteilung2_head_audio.mp4 output Normalverteilung2_merged_start.mp4


frame=    1 fps=0.0 q=-1.0 Lsize=    1433kB time=00:00:59.98 bitrate= 195.7kbits/s speed= 342x    
video:14kB audio:1406kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.872256%


# Concat to endproduct

In [167]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue
        
        #define input for the text file with the videos we want to concat
        video1 = f"{video_name.split('.')[0]}_merged_start.mp4"
        video2 = f"{video_name.split('.')[0]}_ohne_start_standbild.mp4"
        textfile_content = f"file '{video1}'\nfile '{video2}'"

        #create the file with content
        with open('dummy.txt', 'w') as textfile:
            textfile.write(textfile_content)

        #define parameters for concat function
        output_file = f"{video_name.split('.')[0]}_standbild_endprodukt.mp4"
        textfile_name = "dummy.txt"
        print("file content:", textfile_content)
        concatenate_videos(textfile_name,output_file)
        
except Exception as error:
    print("An error occured:", error)

file content: file 'Normalverteilung_merged_start.mp4'
file 'Normalverteilung_ohne_start_standbild.mp4'


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

file content: file 'Normalverteilung2_merged_start.mp4'
file 'Normalverteilung2_ohne_start_standbild.mp4'


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7f8c61712000] Auto-inserting h264_mp4toannexb bitstream filter
Input #0, concat, from 'dummy.txt':
  Duration: N/A, start: 0.000000, bitrate: 3653 kb/s
  Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt709, progressive), 1920x1080 [SAR 1:1 DAR 16:9], 3461 kb/s, 30 fps, 30 tbr, 15360 tbn
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc60.3.100 libx264
  Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 191 kb/s
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]
Output #0, mp4, to 'Normalverteilung2_standbild_endprodukt.mp4':
  Metadata:
    encoder         : Lavf60.3.100
  Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt709, progressive), 1920x1080 [SAR 1:1 DAR 16:9], q=2-31, 3461 kb/s, 30 fps, 30 tbr, 15360 tbn
    Metadata:
      handler_name    : VideoHandler
      vendor_id  

# Create folder and move files

In [168]:
path_folder_endprodukte = "endprodukte"
create_folder(path_folder_endprodukte)

path_folder_script_output = "script_output"
create_folder(path_folder_script_output)

Folder 'endprodukte' created successfully.
Folder 'script_output' created successfully.


In [169]:
original_video_names = list()

for video_name in video_schnitt_df["dateiname"]:
    if video_name != "nan":
        original_video_names.append(video_name)

In [170]:
for filename in os.listdir():
    if filename.endswith("endprodukt.mp4"):
        shutil.move(filename,path_folder_endprodukte)
    elif filename.endswith("mp4") and filename not in original_video_names:
        shutil.move(filename,path_folder_script_output)

# Start to cut things out

In [171]:
# cut all the videos that did not use the standbild ersetzen script
try:
    # videos need to be in same directory with python script
    trash_list = []
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if not os.path.exists(video_name):
            print("Following file does not exist:",video_name)
            continue

        #check ob video schon standbid ersetzt bekommen, use other videoname
        if video_schnitt_df["standbild_bis"][idx] != "nan":
            print(video_name,"skipped because other video in endprodukte needs to be used")
            continue

        columns_to_cut = get_columns_to_cut(video_schnitt_df)
        evens = get_list_of_evens(columns_to_cut)

        parts_list = []
        video1 = 0
        video2 = 0
        end_reached = None
        print("Video Name to cut:", video_name)

        try:
            for i,col in enumerate(columns_to_cut):
                cut_head = video_schnitt_df[col][idx]
                cut_tail = video_schnitt_df[columns_to_cut[i+1]][idx]
                print("index:",i,"Column:",col)
                print("Timestamps before changing:",cut_head,cut_tail)

                #logic to handle the nans 
                
                if cut_head != "nan" and cut_head != "00:00:00" and i == 0:
                    print("Skipped because of special case")
                    
                    break
                
                if cut_head == "nan" and cut_tail == "nan":
                    print("Break used")
                    break
                
                if end_reached == True:
                    break

                if  cut_head == "nan":
                    cut_head = "00:00:00"
                    print("cut_head ersetzt")

                if cut_tail == "nan":
                    cut_tail = get_video_duration(video_name)
                    end_reached = True
                    print("end reached")
                    
                output_file = f"{video_name.split('.')[0]}_part{i}.mp4"
                print("Timestamps after changing:",cut_head,cut_tail)
                cut_head_tail(video_name, output_file, cut_head, cut_tail)
                
                if i not in evens:
                    parts_list.append(output_file)
                else:
                    trash_list.append(output_file)

                if len(parts_list) == 2:
                    video1 = parts_list[0]
                    video2 = parts_list[1]
                    
                    textfile_content = f"file '{video1}'\nfile '{video2}'"

                    #create the file with content
                    with open('dummy.txt', 'w') as textfile:
                        textfile.write(textfile_content)

                    #define parameters for concat function
                    textfile_name = "dummy.txt"

                    output_file = f"{video_name.split('.')[0]}_concatted{i}.mp4"
                    
                    parts_list.clear()
                    parts_list.append(output_file)
                    print(textfile_content, "output:", output_file)
                    concatenate_videos(textfile_name,output_file)                

        except Exception as e:
            print("skip to next video because:",e)

except Exception as error:
    print("An error occured:", error)   

Normalverteilung.mp4 skipped because other video in endprodukte needs to be used
Video Name to cut: ELR_Geradengleichung.mp4
index: 0 Column: rausschneiden1_ab
Timestamps before changing: nan 00:01:44.000
cut_head ersetzt
Timestamps after changing: 00:00:00 00:01:44.000


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

index: 1 Column: rausschneiden1_bis
Timestamps before changing: 00:01:44.000 00:02:04.000
Timestamps after changing: 00:01:44.000 00:02:04.000


frame=  422 fps=0.0 q=-1.0 Lsize=     613kB time=00:00:19.98 bitrate= 251.4kbits/s speed= 243x    
video:127kB audio:468kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 2.964017%
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --en

index: 2 Column: rausschneiden2_ab
Timestamps before changing: 00:02:04.000 00:02:08.000
Timestamps after changing: 00:02:04.000 00:02:08.000


...

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7fcfea108e00] Auto-inserting h264_mp4toannexb bitstream filter
frame= 6632 fps=0.0 q=-1.0 Lsize=   39141kB time=00:04:00.09 bitrate=1335.5kbits/s speed= 534x    
video:33273kB audio:5624kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.624246%


index: 6 Column: rausschneiden4_ab
Timestamps before changing: 00:08:00 nan
end reached
Timestamps after changing: 00:08:00 00:14:42.93


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

index: 7 Column: rausschneiden4_bis
Timestamps before changing: nan nan
Break used
Normalverteilung2.mp4 skipped because other video in endprodukte needs to be used
Following file does not exist: nan


frame=12026 fps=9892 q=-1.0 Lsize=   12811kB time=00:06:42.85 bitrate= 260.5kbits/s speed= 331x    
video:2936kB audio:9442kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 3.499097%


## cut all the videos that did not use the standbild ersetzen script


In [172]:

try:
    # videos need to be in same directory with python script
    trash_list = []
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if not os.path.exists(video_name):
            print("Following file does not exist:",video_name)
            continue

        #check ob video schon standbid ersetzt bekommen, use other videoname
        if video_schnitt_df["standbild_bis"][idx] == "nan":
            print(f"skipped this {video_name} because it is already finished")
            continue
        
        folder_name = "endprodukte/"
        new_video_name = folder_name + f"{video_name.split('.')[0]}_standbild_endprodukt.mp4"

        columns_to_cut = get_columns_to_cut(video_schnitt_df)
        evens = get_list_of_evens(columns_to_cut)

        parts_list = []
        video1 = 0
        video2 = 0
        end_reached = None
        print("Video Name to cut:", video_name, "actual input to cut:", new_video_name)

        try:
            for i,col in enumerate(columns_to_cut):
                cut_head = video_schnitt_df[col][idx]
                cut_tail = video_schnitt_df[columns_to_cut[i+1]][idx]
                print("index:",i,"Column:",col)
                print("Timestamps before changing:",cut_head,cut_tail)

                #logic to handle the nans
                if cut_head != "nan" and cut_head != "00:00:00" and i == 0:
                    print("Skipped because of special case")
                    break
                
                if cut_head == "nan" and cut_tail == "nan":
                    print("Break used")
                    break
                
                if end_reached == True:
                    print("Break used")
                    break

                if  cut_head == "nan":
                    cut_head = "00:00:00"
                    print("cut_head ersetzt")

                if cut_tail == "nan":
                    cut_tail = get_video_duration(video_name)
                    end_reached = True
                    print("end reached")
                    
                output_file = f"{video_name.split('.')[0]}_part{i}.mp4"
                print("Timestamps after changing:",cut_head,cut_tail)
                cut_head_tail(new_video_name, output_file, cut_head, cut_tail)
                
                if i not in evens:
                    parts_list.append(output_file)
                else:
                    trash_list.append(output_file)

                if len(parts_list) == 2:
                    video1 = parts_list[0]
                    video2 = parts_list[1]
                    
                    textfile_content = f"file '{video1}'\nfile '{video2}'"

                    #create the file with content
                    with open('dummy.txt', 'w') as textfile:
                        textfile.write(textfile_content)

                    #define parameters for concat function
                    textfile_name = "dummy.txt"

                    output_file = f"{video_name.split('.')[0]}_concatted{i}.mp4"
                    
                    parts_list.clear()
                    parts_list.append(output_file)
                    print(textfile_content, "output:", output_file)
                    concatenate_videos(textfile_name,output_file)                

        except Exception as e:
            print("skip to next video because:",e)

except Exception as error:
    print("An error occured:", error)   

Video Name to cut: Normalverteilung.mp4 actual input to cut: endprodukte/Normalverteilung_standbild_endprodukt.mp4
index: 0 Column: rausschneiden1_ab
Timestamps before changing: 00:00:00 00:00:00
Timestamps after changing: 00:00:00 00:00:00


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occurred: Command '['ffmpeg', '-i', 'endprodukte/Normalverteilung_standbild_endprodukt.mp4', '-ss', '00:00:00', '-to', '00:00:00', '-c:v', 'copy', '-c:a', 'copy', 'Normalverteilung_part0.mp4']' returned non-zero exit status 1.
index: 1 Column: rausschneiden1_bis
Timestamps before changing: 00:00:00 00:02:00
Timestamps after changing: 00:00:00 00:02:00


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'endprodukte/Normalverteilung_standbild_endprodukt.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf60.3.100
  Duration: 00:50:46.09, start: 0.000000, bitrate: 267 kb/s
  Stream #0:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt709, progressive), 1920x1080 [SAR 1:1 DAR 16:9], 66 kb/s, 29.40 fps, 30 tbr, 15360 tbn (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc60.3.100 libx264
  Stream #0:1[0x2](und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 191 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]
Output #0, mp4, to 'Normalverteilung_part1.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf60.3.100
  Stream #0:

index: 2 Column: rausschneiden2_ab
Timestamps before changing: 00:02:00 00:03:00
Timestamps after changing: 00:02:00 00:03:00


...

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

file 'Normalverteilung_part1.mp4'
file 'Normalverteilung_part3.mp4' output: Normalverteilung_concatted3.mp4


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7f86f501e800] Could not find codec parameters for stream 0 (Video: h264 (avc1 / 0x31637661), none(tv, bt709), 1920x1080, 68 kb/s): unspecified pixel format
Consider increasing the value for the 'analyzeduration' (0) and 'probesize' (5000000) options
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7f86f501e800] Auto-inserting h264_mp4toannexb bitstream filter


index: 4 Column: rausschneiden3_ab
Timestamps before changing: nan nan
Break used
skipped this ELR_Geradengleichung.mp4 because it is already finished
skipped this ELR_Geradengleichung_Einleitung.mp4 because it is already finished
Video Name to cut: Normalverteilung2.mp4 actual input to cut: endprodukte/Normalverteilung2_standbild_endprodukt.mp4
index: 0 Column: rausschneiden1_ab
Timestamps before changing: nan nan
Break used
Following file does not exist: nan


frame=87557 fps=19718 q=-1.0 Lsize=   97592kB time=00:49:46.05 bitrate= 267.7kbits/s speed= 672x    
video:24451kB audio:69986kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 3.340658%


# Rename the right videos to _endprodukt

In [173]:
# create a list with videonames of the youngest version that is not in trash list
files = []

for video_name in video_schnitt_df["dateiname"]:
    original_prefix = video_name.split('.')[0]
    youngest_creation_time = None
    youngest_video = None 
    pattern = pattern = re.compile(rf'^{re.escape(original_prefix)}_part\d+.*$')


    for f in os.listdir():     
        if f.endswith(".mp4") and f not in trash_list and not pattern.match(f):
            if "_" in f:
                part_prefix = remove_end_in_name(f)
            else:
                part_prefix = f.split('.')[0]
            
            if part_prefix == original_prefix:
                creation_time = get_creation_time(f)
                if youngest_creation_time is None or creation_time > youngest_creation_time:
                    youngest_creation_time = creation_time
                    youngest_video = f     
    if youngest_video is not None:
        files.append(youngest_video)    
  
print("All the youngest versions of a video:",files)

All the youngest versions of a video: ['Normalverteilung_concatted3.mp4', 'ELR_Geradengleichung_concatted5.mp4', 'ELR_Geradengleichung_Einleitung_concatted5.mp4', 'Normalverteilung2.mp4']


In [174]:
# create a list of original video_names
original_video_names = list()

for video_name in video_schnitt_df["dateiname"]:
    if video_name != "nan":
        original_video_names.append(video_name)

In [175]:
#Compare the filenames with the original Videos
files_to_reaname = [element for element in files if element not in original_video_names]

print("Files that are gonna be renamed:",files_to_reaname)

Files that are gonna be renamed: ['Normalverteilung_concatted3.mp4', 'ELR_Geradengleichung_concatted5.mp4', 'ELR_Geradengleichung_Einleitung_concatted5.mp4']


In [176]:
# rename the videos
try:
    for file in files_to_reaname:
        new_name = f"{file.split('.')[0]}_cut_things_out_endprodukt.mp4"
        os.rename(file,new_name)
        print(new_name)
except Exception as e:
    print("Error while renaming:",e)

Normalverteilung_concatted3_cut_things_out_endprodukt.mp4
ELR_Geradengleichung_concatted5_cut_things_out_endprodukt.mp4
ELR_Geradengleichung_Einleitung_concatted5_cut_things_out_endprodukt.mp4


# Move files

In [177]:
for filename in os.listdir():
    if filename.endswith("endprodukt.mp4"):
        shutil.move(filename,path_folder_endprodukte)
    elif filename.endswith("mp4") and filename not in original_video_names:
        shutil.move(filename,path_folder_script_output)

# Delete old endproducts

In [178]:
# create a list with videonames of the youngest version that is not in trash list
files = []

for video_name in video_schnitt_df["dateiname"]:
    original_prefix = video_name.split('.')[0]
    youngest_creation_time = None
    youngest_video = None 
    pattern = re.compile(rf'^{re.escape(original_prefix)}_(concatted\d+|standbild).*$')
    print(pattern)
    for f in os.listdir("endprodukte/"):     
    
        if pattern.match(f):
            #print(pattern, "matches", f)
            creation_time = get_creation_time("endprodukte/"+f)
            if youngest_creation_time is None or creation_time > youngest_creation_time:
                youngest_creation_time = creation_time
                youngest_video = f     
    if youngest_video is not None:
        files.append(youngest_video)    
  
print("All the youngest versions of a video:",files)

re.compile('^Normalverteilung_(concatted\\d+|standbild).*$')
re.compile('^ELR_Geradengleichung_(concatted\\d+|standbild).*$')
re.compile('^ELR_Geradengleichung_Einleitung_(concatted\\d+|standbild).*$')
re.compile('^Normalverteilung2_(concatted\\d+|standbild).*$')
re.compile('^nan_(concatted\\d+|standbild).*$')
All the youngest versions of a video: ['Normalverteilung_concatted3_cut_things_out_endprodukt.mp4', 'ELR_Geradengleichung_concatted5_cut_things_out_endprodukt.mp4', 'ELR_Geradengleichung_Einleitung_concatted5_cut_things_out_endprodukt.mp4', 'Normalverteilung2_standbild_endprodukt.mp4']


In [179]:
remove_list = []

for f in os.listdir("endprodukte/"):     
    if f not in files:
        remove_list.append(f)

print(remove_list)

['Normalverteilung_standbild_endprodukt.mp4']


In [180]:
for i in remove_list:
    os.remove("endprodukte/" + i)

In [192]:
folder_path = "endprodukte/"

for video_name in video_schnitt_df["dateiname"]:
    original_prefix = video_name.split('.')[0]
    pattern = re.compile(rf'^{re.escape(original_prefix)}_(concatted\d+|standbild).*$')
    
    for f in os.listdir(folder_path):
        if pattern.match(f):
            new_name = folder_path + original_prefix + ".mp4"
            os.rename(folder_path + f,new_name)

# Start cut_and_keep script

In [ ]:
# 4 Möglichkeiten
# 1. Garnicht beschnitten bisher -> normalzustand
# 2. nur bei standbild endprodukt ->
# 3. cut_things_out_endprodukt ->
# 4. muss nicht beschnitten werden -> video_schnitt_df[col][row] != "nan":

In [220]:
def calculate_timestamp(timestamp):
    colums_to_cut = get_columns_to_cut(video_schnitt_df)
    pattern = re.compile(r'^rausschneiden\d+_bis$')
    for row, video_name in video_schnitt_df["dateiname"].items():
        for col_num,col in enumerate(video_schnitt_df[colums_to_cut]):
            if pattern.match(col):
                print(col)

In [221]:
calculate_timestamp("as")

rausschneiden1_bis
rausschneiden2_bis
rausschneiden3_bis
rausschneiden4_bis
rausschneiden5_bis
rausschneiden6_bis
rausschneiden7_bis
rausschneiden1_bis
rausschneiden2_bis
rausschneiden3_bis
rausschneiden4_bis
rausschneiden5_bis
rausschneiden6_bis
rausschneiden7_bis
rausschneiden1_bis
rausschneiden2_bis
rausschneiden3_bis
rausschneiden4_bis
rausschneiden5_bis
rausschneiden6_bis
rausschneiden7_bis
rausschneiden1_bis
rausschneiden2_bis
rausschneiden3_bis
rausschneiden4_bis
rausschneiden5_bis
rausschneiden6_bis
rausschneiden7_bis
rausschneiden1_bis
rausschneiden2_bis
rausschneiden3_bis
rausschneiden4_bis
rausschneiden5_bis
rausschneiden6_bis
rausschneiden7_bis


In [210]:
#loop through videonames and then through columns to divide
folder_path = "endprodukte/"

for row, video_name in video_schnitt_df["dateiname"].items():
    columns_to_divide = get_columns_to_divide(video_schnitt_df)
    for col_num,col in enumerate(video_schnitt_df[columns_to_divide]):
        
        #rule when there is more then one division
        if video_schnitt_df[col][row] != "nan":
            if col_num > 0:
                input_file = f"{video_name.split('.')[0]}_tail{col_num}.mp4"
                timestamp1 = video_schnitt_df[columns_to_divide[col_num -1]][row]
                timestamp2 = video_schnitt_df[col][row]
                timestamp = calculate_time(timestamp2,timestamp1,kind="subtraction")
                head_output = f"{video_name.split('.')[0]}_mitte{col_num+ 1}.mp4"
                tail_output = f"{video_name.split('.')[0]}_tail{col_num +1}.mp4"
                #print("PRINT: timestamps before calculation:",timestamp1,timestamp2)
                #print("PRINT: all Variables further cut:",input_file,timestamp,head_output,tail_output)

                #cut_in_2_pieces(input_file,timestamp,head_output,tail_output)
                #os.remove(input_file)
            else:

                #define all variables needed
                if video_schnitt_df["standbild_bis"][row] != "nan" or video_schnitt_df["rausschneiden1_bis"][row] != "nan":
                    input_file = folder_path + video_name
                else:
                    input_file = video_name

                if video_schnitt_df["rausschneiden1_bis"][row] != "nan":
                     
                    continue
                    # if there is a timestamp smaller than timestamp_to_divide then we need to calc, else not.
                timestamp = video_schnitt_df[col][row]
                head_output = f"{video_name.split('.')[0]}_head{col_num +1}.mp4"
                tail_output = f"{video_name.split('.')[0]}_tail{col_num +1}.mp4"
                
                #print("all variables:",input_file,timestamp,head_output,tail_output)

                #start seperation
                #cut_in_2_pieces(input_file,timestamp,head_output,tail_output)

endprodukte/Normalverteilung.mp4
endprodukte/ELR_Geradengleichung.mp4
endprodukte/ELR_Geradengleichung_Einleitung.mp4
Normalverteilung2.mp4


In [ ]:
path_folder_endprodukte = "endprodukte"
create_folder(path_folder_endprodukte)

path_folder_script_output = "script_output"
create_folder(path_folder_script_output)

In [ ]:
original_video_names = list()

for video_name in video_schnitt_df["dateiname"]:
    if video_name != "nan":
        original_video_names.append(video_name)

In [ ]:
for filename in os.listdir():
    if filename.endswith("mp4") and filename not in original_video_names:
        shutil.move(filename,path_folder_endprodukte)